In [ ]:
import numpy as np
import matplotlib.pyplot as plt
import os

def read_data(filename):
    data = np.loadtxt(filename)
    x = data[:, 0]
    y = data[:, 1]
    T = data[:, 2]
    return x, y, T


def plot_contours_parallel(tid, px, py):
    # Parameters (adjust according to your simulation)
    NXGLOB = 800   # Global grid size in x
    NYGLOB = 800   # Global grid size in y
    output_dir = './'  # Output directory
    
    # Calculate local grid dimensions
    nx = NXGLOB // px
    ny = NYGLOB // py
    
    # Initialize global temperature array
    T_global = np.zeros((NXGLOB, NYGLOB))
    
    # Process each rank's output file
    for rank in range(px * py):
        # Use the same mapping as in the simulation
        rank_y = rank//px
        rank_x = rank - (rank_y) * px
        
        # Determine global indices for this subdomain
        istart = rank_x * nx
        iend = (rank_x + 1) * nx
        jstart = rank_y * ny
        jend = (rank_y + 1) * ny
        
        # Load data file
        fname = os.path.join(output_dir, f'T_x_y_{tid:06d}_{rank:04d}.dat')
        data = np.loadtxt(fname)
        
        # Reshape data to local grid dimensions (Fortran-order for MATLAB compatibility)
        T_local = data[:, 2].reshape((nx, ny), order='C')

        
        # Insert into global array
        T_global[istart:iend, jstart:jend] = T_local

    # Create coordinate vectors
    x_global = np.linspace(0, 1, NXGLOB)
    y_global = np.linspace(0, 1, NYGLOB)
    X, Y = np.meshgrid(x_global, y_global)
 
    # --- Contour Plot ---
    plt.figure(figsize=(8, 6))
    plt.contourf(X, Y, T_global.T, levels=20, cmap='jet')
    plt.colorbar()
    plt.xlabel('x')
    plt.ylabel('y')
    plt.title(f'Temperature Contours at t = {tid}')
    plt.gca().set_aspect('equal')
    plt.savefig(f'cont_T_{tid:06d}.png', dpi=300, bbox_inches='tight')
    plt.close()

    # --- Midline Profile Plot ---
    plt.figure(figsize=(8, 4))
    mid_idx = NYGLOB // 2
    plt.plot(x_global, T_global[:, mid_idx], 'r', linewidth=2)
    plt.xlabel('x')
    plt.ylabel('Temperature')
    plt.title(f'Midline Profile at t = {tid}')
    plt.grid(True)
    plt.savefig(f'line_midy_T_{tid:06d}.png', dpi=300, bbox_inches='tight')
    plt.close()


plot_contours_parallel(2550, 4, 2)

FileNotFoundError: ./T_x_y_0025500000.dat not found.